In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sbn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from math import ceil

import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
import scipy.stats as scs
import statsmodels.api as sm




from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score


import timeit
import datetime as dt

import pandasql as ps

In [2]:
data = pd.read_csv('~/Downloads/EIA930_BALANCE_2020_Jan_Jun.csv')
data_2 = pd.read_csv('~/Downloads/EIA930_BALANCE_2020_Jul_Dec.csv')

/Users/cp/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (11,14,15,16,17,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/cp/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (11,14,16,17,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def change_cols_to_floats(dataframe,lst):
    
    for i in lst:
        dataframe[i] = dataframe[i].str.replace(',', '')
        dataframe[i] = dataframe[i].astype(float)
    return dataframe
def make_date_time_col(df):
    df['Hour Number'] = df_total['Hour Number'].replace(24, 0)
    df['Hour Number'] = df_total['Hour Number'].replace(25, 0)
    df['Data Date']= df['Data Date'].astype(str)
    df['Hour Number'] = df['Hour Number'].astype(str)
    df['New_datetime'] = df['Data Date'].map(str) + " " + df['Hour Number']
    df['Hour Number'] = df['Hour Number'].astype(int)
    
    return df

def make_hourly_demand_means(df,lst):
    d = {}
    for i in lst:
        filt =df['Hour Number']==i
        d[i] = df.loc[filt]['Demand (MW)'].mean()
    return d

In [4]:
lst_cols = ['Demand (MW)','Net Generation (MW) from Natural Gas', 'Net Generation (MW) from Nuclear','Net Generation (MW) from All Petroleum Products','Net Generation (MW) from Hydropower and Pumped Storage', 'Net Generation (MW) from Solar', 'Net Generation (MW) from Wind', 'Net Generation (MW) from Other Fuel Sources','Net Generation (MW)','Demand Forecast (MW)', 'Total Interchange (MW)', 'Net Generation (MW) (Adjusted)','Net Generation (MW) from Coal','Sum(Valid DIBAs) (MW)','Demand (MW) (Imputed)', 'Net Generation (MW) (Imputed)','Demand (MW) (Adjusted)']
data_convert = change_cols_to_floats(data, lst_cols)
data_2_convert = change_cols_to_floats(data_2, lst_cols)

In [5]:
lst_data = [data_convert,data_2_convert]
df_total = pd.concat(lst_data) 

In [6]:
make_date_time_col(df_total)

,Balancing Authority,Data Date,Hour Number,Local Time at End of Hour,UTC Time at End of Hour,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Sum(Valid DIBAs) (MW),...,Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Net Generation (MW) from Unknown Fuel Sources,New_datetime
0,AEC,01/01/2020,1,01/01/2020 1:00:00 AM,01/01/2020 7:00:00 AM,1082.0,549.0,722.0,173.0,173.0,...,-5.0,722.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,01/01/2020 1
1,AEC,01/01/2020,2,01/01/2020 2:00:00 AM,01/01/2020 8:00:00 AM,1084.0,549.0,724.0,175.0,175.0,...,-3.0,722.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,01/01/2020 2
2,AEC,01/01/2020,3,01/01/2020 3:00:00 AM,01/01/2020 9:00:00 AM,1103.0,554.0,718.0,164.0,164.0,...,-6.0,719.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,01/01/2020 3
3,AEC,01/01/2020,4,01/01/2020 4:00:00 AM,01/01/2020 10:00:00 AM,1127.0,565.0,715.0,150.0,150.0,...,-4.0,714.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,01/01/2020 4
4,AEC,01/01/2020,5,01/01/2020 5:00:00 AM,01/01/2020 11:00:00 AM,1179.0,575.0,718.0,143.0,143.0,...,-4.0,717.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,01/01/2020 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282671,YAD,12/31/2020,20,12/31/2020 8:00:00 PM,01/01/2021 1:00:00 AM,NaN,NaN,154.0,150.0,150.0,...,NaN,NaN,NaN,NaN,154.0,NaN,NaN,NaN,NaN,12/31/2020 20
282672,YAD,12/31/2020,21,12/31/2020 9:00:00 PM,01/01/2021 2:00:00 AM,NaN,NaN,153.0,150.0,150.0,...,NaN,NaN,NaN,NaN,153.0,NaN,NaN,NaN,NaN,12/31/2020 21
282673,YAD,12/31/2020,22,12/31/2020 10:00:00 PM,01/01/2021 3:00:00 AM,NaN,NaN,154.0,150.0,150.0,...,NaN,NaN,NaN,NaN,154.0,NaN,NaN,NaN,NaN,12/31/2020 22
282674,YAD,12/31/2020,23,12/31/2020 11:00:00 PM,01/01/2021 4:00:00 AM,NaN,NaN,154.0,151.0,151.0,...,NaN,NaN,NaN,NaN,154.0,NaN,NaN,NaN,NaN,12/31/2020 23


In [7]:
df_total['New_datetime']= df_total['New_datetime'].apply(lambda x:f'{x}:00:00')

In [8]:
df_total['New_datetime'] = pd.to_datetime(df_total['New_datetime'],infer_datetime_format=True, format ='%m/%d/%Y %H')

In [9]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 562344 entries, 0 to 282675
Data columns (total 24 columns):
 #   Column                                                  Non-Null Count   Dtype         
---  ------                                                  --------------   -----         
 0   Balancing Authority                                     562344 non-null  object        
 1   Data Date                                               562344 non-null  object        
 2   Hour Number                                             562344 non-null  int64         
 3   Local Time at End of Hour                               562344 non-null  object        
 4   UTC Time at End of Hour                                 562344 non-null  object        
 5   Demand Forecast (MW)                                    463182 non-null  float64       
 6   Demand (MW)                                             471116 non-null  float64       
 7   Net Generation (MW)                            

In [10]:
df_total['Hour Number'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23,  0])

In [11]:
df_total['Demand Delta'] = df_total['Demand Forecast (MW)']- df_total['Demand (MW)']

In [12]:
df_total['Net Generation Delta'] = df_total['Net Generation (MW)']- df_total['Demand (MW)']

In [13]:
lst_hours = np.arange(0,24)
# make_hourly_demand_means(df_total, lst_hours)


df_total['Hour Number'].unique()

make_hourly_demand_means(df_total, lst_hours)

{0: 6922.334387109932,
 1: 6468.399511301161,
 2: 7152.640769779045,
 3: 6998.285416030146,
 4: 6080.086872390264,
 5: 6139.650081482991,
 6: 7206.099302337424,
 7: 6713.549602770422,
 8: 7886.163865974132,
 9: 7288.728217141112,
 10: 8347.805948864216,
 11: 8533.530891865736,
 12: 8727.126273431133,
 13: 8886.620427916454,
 14: 9009.337477073568,
 15: 7399.706598726115,
 16: 9188.23905620955,
 17: 9310.143082802548,
 18: 8617.770575345257,
 19: 9469.125522372848,
 20: 9308.599133537207,
 21: 6532.888237692386,
 22: 7189.769430711992,
 23: 7388.96743119266}

In [14]:
average_hourly_demand = df_total.groupby(['Hour Number']).mean('Demand (MW)')
average_hourly_demand.head()

,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Sum(Valid DIBAs) (MW),Demand (MW) (Imputed),Net Generation (MW) (Imputed),Demand (MW) (Adjusted),Net Generation (MW) (Adjusted),Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Net Generation (MW) from Unknown Fuel Sources,Demand Delta,Net Generation Delta
Hour Number,,,,,,,,,,,,,,,,,,,,
0,7765.880517,6922.334387,6373.218225,689.819066,734.902239,2062.515723,2741.232394,7741.012509,6337.306171,2323.051155,2932.107716,5242.341601,277.910606,579.979005,1.323596,1592.796856,271.300757,NaN,-119.498800,573.701485
1,7368.957082,6468.399511,6059.127887,745.072253,1449.950562,2067.593985,3487.153061,7377.259126,6042.660637,2177.189058,2726.078609,5247.287508,252.124440,526.717212,0.622480,1575.837625,262.581474,NaN,-51.769299,670.426950
2,7121.973943,7152.640770,5859.195241,-39.922558,-1.869050,2564.985075,3777.897959,7129.337502,5848.345136,2079.410427,2593.297112,5243.947190,233.942201,501.206012,0.371479,1551.000000,258.320938,NaN,-138.129815,-252.024997
3,6984.869727,6998.285416,5744.138132,-32.801976,1.839821,2437.102190,3625.524272,6977.670852,5730.691127,2021.394308,2526.897637,5242.784932,225.388654,487.925380,0.310328,1516.946769,254.430913,NaN,-120.579901,-232.535340
4,6946.112337,6080.086872,5701.911239,685.856449,714.721200,2390.323741,3542.813084,6910.771404,5685.654650,2002.072918,2512.169684,5244.223760,222.665699,488.053464,0.277880,1483.318881,251.776601,NaN,-91.381912,633.925960


In [16]:
df_total.head()

,Balancing Authority,Data Date,Hour Number,Local Time at End of Hour,UTC Time at End of Hour,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Sum(Valid DIBAs) (MW),...,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Net Generation (MW) from Unknown Fuel Sources,New_datetime,Demand Delta,Net Generation Delta
0,AEC,01/01/2020,1,01/01/2020 1:00:00 AM,01/01/2020 7:00:00 AM,1082.0,549.0,722.0,173.0,173.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2020-01-01 01:00:00,533.0,173.0
1,AEC,01/01/2020,2,01/01/2020 2:00:00 AM,01/01/2020 8:00:00 AM,1084.0,549.0,724.0,175.0,175.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2020-01-01 02:00:00,535.0,175.0
2,AEC,01/01/2020,3,01/01/2020 3:00:00 AM,01/01/2020 9:00:00 AM,1103.0,554.0,718.0,164.0,164.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2020-01-01 03:00:00,549.0,164.0
3,AEC,01/01/2020,4,01/01/2020 4:00:00 AM,01/01/2020 10:00:00 AM,1127.0,565.0,715.0,150.0,150.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2020-01-01 04:00:00,562.0,150.0
4,AEC,01/01/2020,5,01/01/2020 5:00:00 AM,01/01/2020 11:00:00 AM,1179.0,575.0,718.0,143.0,143.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2020-01-01 05:00:00,604.0,143.0


# EDA

In [ ]:
sample_data = df_total.sample(n =500)

In [ ]:
sample_data['Demand Delta'] = sample_data['Demand Forecast (MW)']- sample_data['Demand (MW)']

In [ ]:
sample_data['Net Generation Delta'] = sample_data['Net Generation (MW)']- sample_data['Demand (MW)']

In [ ]:
average_hourly_demand = sample_data.groupby(['Hour Number']).mean('Demand (MW)')

In [ ]:
filt =sample_data['Hour Number']==23
hour1 = sample_data.loc[filt]['Demand (MW)'].mean()
print(hour1)

In [ ]:
def make_hourly_demand_means(df,lst):
    d = {}
    for i in lst:
        filt =df['Hour Number']==i
        d[i] = df.loc[filt]['Demand (MW)'].mean()
    return d

In [ ]:
lst_hours = np.arange(0,24)
make_hourly_demand_means(df_total, lst_hours)
# make_hourly_demand_means(sample_data, lst_hours)

In [ ]:
filt =df_total['Hour Number']==23
hour1 = df_total.loc[filt]['Demand (MW)'].mean()
print(hour1)

In [ ]:
def func2(df, lst):
    d= {}
    for i in lst:
        filt =df['Hour Number']==i
        hour_avg = df.loc[filt]['Demand (MW)'].mean()
        d[i]=hour_avg
    return d

        
def make_hourly_demand_means(df,lst):
    d = {}
    for i in lst:
        filt =df['Hour Number']==i
        d[i] = df.loc[filt]['Demand (MW)'].mean()
    return d      
        

In [ ]:
make_hourly_demand_means(sample_data, lst_hours)

In [ ]:
demand_dct = func2(df_total, lst_hours)

In [ ]:
x = demand_dct.keys()
y =demand_dct.values()


In [ ]:
fig, ax =plt.subplots(figsize = (12,12))
ax.plot(x, y)
plt.show()

In [ ]:
def func3(df,column, lst = np.arange(0,24)):
    
    d= {}
    for i in lst:
        filt =df['Hour Number']==i
        hour_avg = df.loc[filt][column].mean()
        d[i]=hour_avg
    x = d.keys()
    y = d.values()
    fig, ax =plt.subplots(figsize = (8,8))
    ax.plot(x, y)
    ax.set_title(column)
    ax.set_xlabel('Hours in Day')
    ax.set_xticks(lst)
    
    
    plt.show()
#     return d

In [ ]:
func3(df_total,'Net Generation (MW) from Nuclear' )

In [ ]:
# x_nuc = demand_dct.keys()
# y_nuc =demand_dct.values()

In [ ]:
func3(df_total,'Net Generation (MW) from Wind' )

In [ ]:
func3(df_total,'Net Generation (MW) from Solar' )

In [ ]:
func3(df_total,'Net Generation (MW)' )

In [ ]:
func3(df_total,'Demand (MW)' )

In [ ]:
func3(df_total,'Demand Delta' )

In [ ]:
func3(df_total,'Net Generation Delta')

In [ ]:
sample_data.head()
sample_data['Balancing Authority'].unique()

In [ ]:
df_total.head()


# Texas ERCO

In [ ]:
filter_1 = df_total['Balancing Authority'] == 'ERCO'
df_texas = df_total[filter_1]
df_texas


# df_total.query('Balancing Authority' == 'ERCO')
# data.query('Senior_Management == True 
#             and Gender =="Male" and Team =="Marketing" 
#             and First_Name =="Johnny"', inplace = True) 

In [ ]:
func3(df_texas, 'Demand (MW)')

In [ ]:
func3(df_texas, 'Net Generation (MW) from Solar')

In [ ]:
func3(df_texas, 'Demand Delta')

In [ ]:
func3(df_texas, 'Net Generation Delta')

In [ ]:
df_texas.columns
catagories_lst = ['Demand Forecast (MW)''Net Generation (MW) (Imputed)',
       'Demand (MW) (Adjusted)', 'Net Generation (MW) (Adjusted)',
       'Net Generation (MW) from Coal', 'Net Generation (MW) from Natural Gas',
       'Net Generation (MW) from Nuclear',
       'Net Generation (MW) from All Petroleum Products',
       'Net Generation (MW) from Hydropower and Pumped Storage',
       'Net Generation (MW) from Solar', 'Net Generation (MW) from Wind','Demand Delta', 'Net Generation Delta']

In [ ]:



def func_tex(df,catagory_lst, rang = np.arange(0,24)):
#      d = {}
#     for i in catagory_lst:
#         if i not in d:
#             d[i]={}
            
    d = dct_maker(df_total,catagory_lst, lst = np.arange(0,24))
    for item in catagory_lst:
        for i in rang:
            filt =df['Hour Number']==i
            hour_avg = df.loc[filt][item].mean()
            item[i]=hour_avg
            
            
        x = item.keys()
        y = item.values()
    fig, ax =plt.subplots(figsize = (8,8))
    ax.plot(x, y)
    ax.set_title('energy sources')
    ax.set_xlabel('Hours in Day')
    ax.set_xticks(lst)
    return d
# func_tex(df_total,catagories_lst, rang = np.arange(0,24))

In [ ]:
def dct_maker(df,catagory_lst, lst = np.arange(0,24)):
    d = {}
    for i in catagory_lst:
        if i not in d:
            d[i]={}
    return d
dct_maker(df_total,catagories_lst, lst = np.arange(0,24))

In [ ]:
def func4(df,column, lst = np.arange(0,24)):
    
    
    Demand_Forecast={}
    Net_Generation={}
    Demand={}
    Net_Generation_Adjusted={}
    Net_Generation_Coal={} 
    Net_Generation_Natural_Gas={}
    Net_Generation_Nuclear={}
    Net_Generation_Petroleum={}
    Net_Generation_Hydropower={}
    Net_Generation_Solar={}
    Net_Generation_Wind={}
    Demand_Delta={}
    Net_Generation_Delta={}
    
   
    
    for i in lst:
        filt =df['Hour Number']==i
        hour_avg = df.loc[filt][column].mean()
        d[i]=hour_avg
    x = d.keys()
    y = d.values()
    fig, ax =plt.subplots(figsize = (8,8))
    ax.plot(x, y)
    ax.plot(x1, y1)
    ax.set_title(column)
    ax.set_xlabel('Hours in Day')
    ax.set_xticks(lst)
    
    
    plt.show()

In [ ]:
def func_tester(df,column, lst = np.arange(0,24)):
    fig, ax =plt.subplots(figsize = (8,8))
  
    
    for item in col_list:
        for i in lst:
            d= {}
            filt =df['Hour Number']==i
            hour_avg = df.loc[filt][column].mean()
            d[i]=hour_avg
            x = d.keys()
            y = d.values()

            ax.plot(x, y)
#             d = {}
        ax.set_title(column)
        ax.set_xlabel('Hours in Day')
        ax.set_xticks(lst)

    
    plt.show()
#     return d

In [ ]:
df_texas
del df_texas['UTC Time at End of Hour']
del df_texas['Balancing Authority']
del df_texas['Net Generation (MW) (Imputed)']
df_texas

In [ ]:
df_texas

In [ ]:
df_texas['Balancing Authority'].unique()

In [ ]:
df_texas.info()

In [ ]:
df_texas.info()

In [ ]:
df_texas.shape

In [ ]:
df_texas['Data Date'].value_counts()

In [ ]:
df_texas.to_csv (r'/Users/cp/Desktop/capstone2/df_texas_cleanv1.csv', index = False, header=True)

In [ ]:
converted_houston_temp_24 = pd.read_csv ('/Users/cp/Desktop/capstone2/Houston_weather_hours.csv')

In [ ]:
# result = pd.merge(df_texas, converted_houston_temp_24, on =['New_datetime', 'New_datetime'])
df_texas.join(converted_houston_temp_24, on ='New_datetime', how = 'left')

In [ ]:
converted_houston_temp_24.info()

In [ ]:
converted_houston_temp_24

In [ ]:
converted_houston_temp_24['New_datetime'] = pd.to_datetime(converted_houston_temp_24['New_datetime'],infer_datetime_format=True, format ='%m/%d/%Y %H')

In [ ]:
df_texas.join(converted_houston_temp_24, on ='New_datetime', how = 'left')

In [ ]:
converted_houston_temp_24.info()

In [ ]:
df_texas.info()

In [ ]:
converted_houston_temp_24.info()

In [ ]:
df_texas.join(converted_houston_temp_24, on ='New_datetime', how = 'left')

In [ ]:
Energy_Houston_weather=df_texas.merge(converted_houston_temp_24, left_on ='New_datetime', right_on='New_datetime' )

In [ ]:
Energy_Houston_weather

In [ ]:
Energy_Houston_weather.to_csv (r'/Users/cp/Desktop/capstone2/Energy_Houston_weather.csv', index = False, header=True)

In [ ]:
Energy_Houston_weather.columns

In [ ]:
Energy_Houston_weather.info()

In [ ]:
df_texas['Net Generation Delta']